In [ ]:
%matplotlib inline
%reload_ext autoreload
%autoreload 2

In [ ]:
ls

In [ ]:
from fastai.structured import *
from fastai.column_data import *

PATH='../input/'

In [ ]:
df = pd.read_csv('{}train.csv'.format(PATH), engine='python')

In [ ]:
df_test_csv = pd.read_csv('{}test.csv'.format(PATH), engine='python')

In [ ]:
df_test_csv['winPlacePerc'] = 0

In [ ]:
df.head()

In [ ]:
df['groupId']

In [ ]:
# arr1 = [1,2,3,4,5]
# arr2 = [6,7,8,9,10]
# arr_idx = [1,3,5]
# list(map(lambda x, idx: sum(x) if idx not in arr_idx else x[0], list(zip(arr1, arr2)), list(range(len(arr1)))))

In [ ]:
df = df.sort_values(by='groupId')

In [ ]:
columns = df.columns.tolist()

In [ ]:
ignore_idx = [0, 1, 2, 14, 15, 20, 25]
list(enumerate(columns))

In [ ]:
values = df.values

In [ ]:
values[0]

In [ ]:
dic = dict()
def dict_func(arr1, arr2):
    return list(map(lambda x, idx: sum(x) if idx not in ignore_idx else x[0], list(zip(arr1, arr2)), list(range(len(arr1)))))

In [ ]:
for entry in values:
    if entry[1] in dic:
        dic.update({entry[1]: dict_func(dic[entry[1]], entry)})
    else:
        dic[entry[1]] = list(entry)

In [ ]:
max(dic.keys())

In [ ]:
dict_values = list(dic.values())

In [ ]:
dict_values

In [ ]:
processed_df = pd.DataFrame(data=dict_values, columns=columns)

In [ ]:
processed_df.head()

In [ ]:
def mae(y_pred, targ):
    return sum(list(map(lambda x: torch.abs(x[0] - x[1]), zip(y_pred, targ)))) / len(y_pred)

In [ ]:
to_drop = ['Id', 'groupId', 'matchId', 'headshotKills', 'longestKill', 'maxPlace', 'swimDistance']
df, y, nas, mapper = proc_df(processed_df, 'winPlacePerc', do_scale=True, skip_flds=to_drop)

In [ ]:
df_test, _, nas, mapper = proc_df(df_test_csv, 'winPlacePerc', do_scale=True, skip_flds=to_drop,
                                  mapper=mapper, na_dict=nas)

In [ ]:
df_test.head()

In [ ]:
train_ratio = 0.7
train_size = int(len(df) * train_ratio)
val_idx = list(range(train_size, len(df)))

In [ ]:
md = ColumnarModelData.from_data_frame('.', val_idx, df, y.astype(np.float32), cat_flds=[], bs=128, test_df=df_test)

In [ ]:
m0 = md.get_learner([], len(df.columns), 0.04, 1, [1500, 750], [0.001, 0.01], y_range=[0, 1])

In [ ]:
m0.summary()

In [ ]:
m0.lr_find()

In [ ]:
m0.sched.plot()

In [ ]:
lr = 1.5 * 1e-5
from sklearn.metrics import mean_squared_error

In [ ]:
m0.fit(lr, 3, metrics=[mae])

In [ ]:
m0.fit(lr, 3, metrics=[mae], cycle_len=2)

In [ ]:
#m0.fit(lr, 3, metrics=[mae], cycle_len=2)

In [ ]:
m0.save('mae058')

In [ ]:
m0.load('mae058')

In [ ]:
x,y=m0.predict_with_targs()

In [ ]:
pred_test=m0.predict(True)

In [ ]:
pred_test

In [ ]:
results_df = pd.DataFrame(columns=['Id', 'winPlacePerc'])

In [ ]:
results_df.winPlacePerc = [x[0] for x in pred_test]
results_df.Id = df_test_csv.Id

In [ ]:
results_df.to_csv('results.csv', index=False)

In [ ]:
df_test.head()

In [ ]:
df.head()